# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Lower Grade Glioma"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [5]:
trait_subdir = "TCGA_Lower_Grade_Glioma_(LGG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LGG.sampleMap_LGG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LGG.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'additional_surgery_metastatic_procedure',
 'age_at_initial_pathologic_diagnosis']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history', 'animal_insect_allergy_types', 'asthma_history', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_additional_surgery_locoregional_procedure', '

In [12]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [31.0, 67.0, 44.0, 37.0, 50.0],
 'days_to_birth': [-11509.0, -24578.0, -16297.0, -13565.0, -18494.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, 234.0, 1335.0, 1106.0, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [3574.0, nan, nan, 481.0, 1828.0],
 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan],
 'first_diagnosis_age_of_animal_insect_allergy': ['12 - 20 Years',
  nan,
  nan,
  nan,
  nan],
 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan],
 'year_of_initial_pathologic_diagnosis': [2005.0,
  2005.0,
  2006.0,
  2009.0,
  2010.0]}

In [14]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [15]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [16]:
gender_col = 'gender'

In [17]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [18]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [19]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lower Grade Glioma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-CS-4938-01,1,31.0,0.0,1.675408,2.256574,-0.025535,-0.129672,0.512522,1.00569,0.980606,...,2.145814,-0.086682,-0.748878,-0.411617,0.508467,-1.28139,0.587923,-1.931327,-1.200995,-5.525533
TCGA-CS-4941-01,1,67.0,1.0,1.677108,1.531874,-0.103335,-0.044972,1.044022,0.68629,0.715906,...,3.563814,-0.086682,2.294922,2.043683,0.703167,-1.28139,-0.211777,-0.691827,-0.067395,-4.274433
TCGA-CS-4942-01,1,44.0,0.0,1.451408,0.339574,0.227665,-0.359972,0.479422,0.83969,0.928806,...,2.061014,-0.086682,-0.748878,2.006583,1.287867,-1.28139,-0.521377,-1.996927,-1.020895,-5.096833
TCGA-CS-4943-01,1,37.0,1.0,1.159508,1.747574,0.193265,0.049128,0.345022,0.20049,0.736706,...,-0.864186,-0.086682,-0.748878,-0.862817,0.231767,-1.28139,0.760023,-1.260727,-2.100995,-4.541333
TCGA-CS-4944-01,1,50.0,1.0,0.847608,3.430774,-0.531035,-0.503272,0.820222,1.21089,0.934706,...,3.457214,-0.086682,-0.159678,1.123883,0.491667,-1.28139,-0.713077,-1.252627,-1.109495,-3.516133


In [20]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 529 samples.


In [21]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lower Grade Glioma', the least common label is '1' with 529 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Lower Grade Glioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 32.0
  50% (Median): 41.0
  75%: 53.0
Min: 14.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 238 occurrences. This represents 44.99% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [22]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [23]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [24]:
dataset = 'GEO'
trait_subdir = "Lower-Grade-Glioma"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE74567',
 'GSE4058',
 'GSE28271',
 'GSE143843',
 'GSE35158',
 'GSE111627',
 'GSE19578',
 'GSE107850',
 'GSE24072',
 'GSE14',
 'GSE145510',
 'GSE15309']

Repeat the below steps for all the accession numbers

In [145]:
cohort = accession_num = "GSE15309"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/content/drive/MyDrive/AI4Science_Public/data/GEO/Lower-Grade-Glioma/GSE15309/GSE15309_family.soft.gz',
 '/content/drive/MyDrive/AI4Science_Public/data/GEO/Lower-Grade-Glioma/GSE15309/GSE15309_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [146]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Cancer Genome Anatomy Project (CGAP) Human SAGE"
!Series_summary	"This series represents the Cancer Genome Anatomy Project SAGE library collection.  Libraries contained herein were either produced through CGAP funding, or donated to CGAP."
!Series_summary	"The Cancer Genome Anatomy Project (CGAP: http://cgap.nci.nih.gov) is an interdisciplinary program established and administered by the National Cancer Institute (NCI: http://www.nci.nih.gov) to generate the information and technological tools needed to decipher the molecular anatomy of the cancer cell."
!Series_summary	""
!Series_summary	"SAGE libraries are named according to the following convention:"
!Series_summary	"* SAGE_Organ_histology_code_unique identifier, e.g., SAGE_Colon_adenocarcinoma_CL_Caco2"
!Series_summary	"* Codes: B = bulk; CL = cell line; CS = short-term cell culture; MD = micro-dissected; AP = antibody purified."
!Series_overall_design	"324 libraries"


In [147]:
clinical_data.head()

,!Sample_geo_accession,GSM383679,GSM383680,GSM383682,GSM383683,GSM383684,GSM383686,GSM383687,GSM383688,GSM383689,...,GSM384027,GSM384028,GSM384029,GSM384030,GSM384031,GSM384032,GSM384033,GSM384034,GSM384035,GSM384036
0,!Sample_characteristics_ch1,organ/tissue: bone marrow,organ/tissue: bone marrow,organ/tissue: bone marrow,organ/tissue: bone marrow,organ/tissue: bone marrow,organ/tissue: brain,organ/tissue: brain,organ/tissue: brain,organ/tissue: brain,...,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells,organ/tissue: white blood cells
1,!Sample_characteristics_ch1,tissue preparation: flow-sorted,tissue preparation: flow-sorted,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: bulk,tissue preparation: cell line,tissue preparation: bulk,tissue preparation: bulk,tissue preparation: bulk,...,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified,tissue preparation: antibody purified
2,!Sample_characteristics_ch1,sex: unknown,sex: unknown,sex: unknown,sex: unknown,sex: unknown,age: fetal,age: 34,age: 35 yrs,age: 23,...,age: 45,age: 71,age: 45,age: 71,age: 39,age: 68,age: 72,age: 31,age: 71,age: 72
3,!Sample_characteristics_ch1,"cell type: Stem Proginitor Cells, CD34+, CD38+",cell type: Acute myelogenous leukemia,cell type: normal bone marrow,cell type: normal bone marrow,NaN,"cell type: normal fetal astrocyte, GFAP positi...",sex: male,sex: male,sex: male,...,sex: male,sex: male,sex: male,sex: male,sex: male,sex: female,sex: female,sex: female,sex: male,sex: male
4,!Sample_characteristics_ch1,NaN,NaN,NaN,NaN,NaN,other information: Normal human astrocyte cell...,"cell type: pilocytic astrocyoma, cerebellum",cell type: astrocytoma grade II,cell type: astrocytoma grade II,...,cell type: monocyte-depleted mononuclear cells,cell type: monocyte-depleted mononuclear cells,cell type: monocyte,cell type: monocyte,cell type: monocyte,cell type: monocyte,cell type: monocyte,cell type: normal leukocytes,cell type: plaque macrophage,cell type: plaque macrophage


In [148]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['organ/tissue: bone marrow',
  'organ/tissue: brain',
  'organ/tissue: cerebellum',
  'organ/tissue: mammary gland',
  'organ/tissue: cartilage',
  'organ/tissue: colon',
  'organ/tissue: stem cell',
  'organ/tissue: other',
  'organ/tissue: eye',
  'organ/tissue: uncharacterized tissue',
  'organ/tissue: heart',
  'organ/tissue: kidney',
  'organ/tissue: white blood cells',
  'organ/tissue: liver',
  'organ/tissue: lung',
  'organ/tissue: lymph node',
  'organ/tissue: muscle',
  'organ/tissue: ovary',
  'organ/tissue: pancreas',
  'organ/tissue: bone',
  'organ/tissue: peritoneum',
  'organ/tissue: placenta',
  'organ/tissue: prostate',
  'organ/tissue: retina',
  'organ/tissue: skin',
  'organ/tissue: spinal cord',
  'organ/tissue: stomach',
  'organ/tissue: gastrointestinal tract',
  'organ/tissue: testis',
  'organ/tissue: thyroid',
  'organ/tissue: uterus',
  'organ/tissue: vascular'],
 1: ['tissue preparation: flow-sorted',
  'tissue preparation: antibody purified',
  'tissu

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [149]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Lower Grade Glioma\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Lower Grade Glioma\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. Th

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [150]:
is_gene_availabe = False
trait_row_id = 3
age_row_id = 2
gender_row_id = 2

trait_type = 'binary'

In [151]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [116]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    if 'glioma grade' in value:
        return 1
    else:
        return None


def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format



def convert_gender(value):
    # For 'gender': Binary
    if 'female' in value.lower():
        return '1'
    elif 'male' in value.lower():
        return '0'
    else:
        return None  # Unknown gender



In [117]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM590744,GSM590745,GSM590746,GSM590747,GSM590748,GSM591700,GSM591701,GSM591702,GSM591703,GSM591704,...,GSM591812,GSM591815,GSM591817,GSM591818,GSM591819,GSM591829,GSM591830,GSM591831,GSM591832,GSM591833
Lower Grade Glioma,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Age,72,70,34,54,34,68,30,60,73,52,...,69,36,38,63,74,67,54,46,55,72
Gender,0,1,0,0,0,0,1,0,0,1,...,1,1,1,0,0,1,0,0,1,0


### Genetic data preprocessing and final filtering

In [118]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM590744,GSM590745,GSM590746,GSM590747,GSM590748,GSM591700,GSM591701,GSM591702,GSM591703,...,GSM591812,GSM591815,GSM591817,GSM591818,GSM591819,GSM591829,GSM591830,GSM591831,GSM591832,GSM591833
0,1007_s_at,3.642191,3.701979,3.549620,3.668898,3.646627,3.717462,3.682097,3.667317,3.641540,...,3.662805,3.677109,3.647491,3.583830,3.605021,3.490284,3.574239,3.557329,3.643160,3.618054
1,1053_at,3.024226,3.072491,3.178727,2.938471,2.975731,3.053342,3.074540,3.100501,3.124698,...,3.169326,3.138912,3.034664,3.027387,3.170743,3.152728,2.986613,2.884332,3.196361,3.169115
2,117_at,3.301488,2.976024,3.064205,2.390648,3.175775,3.036856,2.890154,2.939648,2.939885,...,2.980282,2.941577,2.948396,3.127264,2.842070,2.998904,2.879601,2.918557,2.970701,2.860472
3,121_at,3.326044,3.210726,3.272794,3.345455,3.477646,3.229819,3.343052,3.398781,3.324753,...,3.315291,3.318474,3.438588,3.484291,3.390079,3.380583,3.377112,3.342801,3.411945,3.367096
4,1255_g_at,2.699437,2.523530,2.769053,2.914576,2.695694,2.660008,2.538062,2.724338,2.828592,...,2.640814,3.087213,2.715261,2.945468,2.778569,2.793341,2.852065,2.653127,2.894916,2.495179


In [119]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1598_g_at',
 '160020_at',
 '1729_at',
 '1773_at',
 '177_at',
 '179_at',
 '1861_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [120]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at', '179_at', '1861_at']\n"

If not required, jump directly to the gene normalization step

In [121]:
requires_gene_mapping = True

In [122]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [123]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Databa

In [124]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [125]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [126]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [127]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 32 samples.


In [128]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lower Grade Glioma', the least common label is '1' with 32 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Lower Grade Glioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 49.75
  50% (Median): 65.5
  75%: 70.0
Min: 30
Max: 76
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 13 occurrences. This represents 40.62% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [129]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [130]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'Xena'

In [ ]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,1215.0,
3,GSE248830,True,True,False,True,True,38.0,


In [ ]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Breast Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
0,1,55.0,0.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,...,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212
1,1,50.0,0.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,...,0.312922,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812
2,1,62.0,0.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,...,-0.748878,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788
3,1,52.0,0.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,...,0.415422,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888
4,1,50.0,0.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,...,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:
if has_batch_effect:
    model_constructor = VariableSelection
    model_params = {}
else:
    model_constructor = Lasso
    model_params = {'alpha': 1.0, 'random_state': 42}

In [ ]:
cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params, target_type=trait_type)

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 91.28% ± 1.50%


In [ ]:
# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

/home/techt/anaconda3/envs/hl/lib/python3.8/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0065
p-value: nan

Found 76 significant genes associated with the trait 'Breast Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
    Variable  Coefficient  corrected_p_value
     COL10A1     0.136339       3.404090e-27
         BGN     0.095422       1.004817e-14
    PPAPDC1A     0.081296       9.785054e-13
       MMP11     0.080354       1.123019e-10
       WISP1     0.074888       9.613830e-10
     HSD17B6     0.051940       6.980811e-08
      CTHRC1     0.058310       2.210553e-07
       MFAP5    -0.054250       2.706783e-07
      CLEC5A     0.047846       8.877914e-07
   C20orf103     0.048957       8.877914e-07
          GC     0.027375       1.419502e-05
      OR51D1     0.022908       1.644207e-05
        COMP     0.041129       4.366228e-05
LOC100169752     0.019398       4.366228e-05
      LRRC15     0.052746       4.494130e-05
         FN1     0.063527       5.027562e-05
     C

/home/techt/AI4Science_gold_standard/stage1/Haoyang/utils.py:492: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
